In [1]:

#!wget -q https://github.com/DM-GY-9103-2024F-H/9103-utils/raw/main/src/data_utils.py
!wget -q https://github.com/DM-GY-9103-2024F-H/9103-utils/raw/main/src/image_utils.py
#!wget -q https://github.com/IntrotoML9103/Project/blob/main/final_utils.py

#!wget -qO- https://github.com/DM-GY-9103-2024F-H/9103-utils/releases/latest/download/lfw.tar.gz | tar xz

Data Processing

In [ ]:
import os
import random
from os import listdir, path
import torch
from torch import Tensor  
from image_utils import make_image, open_image


random.seed(42)

artist_names = [
    "Albrecht_Dürer",
    "Alfred_Sisley",
    "Amedeo_Modigliani",
    "Andy_Warhol",
    "Diego_Velazquez",
    "Edgar_Degas",
    "Francisco_Goya",
    "Frida_Kahlo",
    "Giotto_di_Bondone",
    "Gustav_Klimt",
    "Henri_Matisse",
    "Hieronymus_Bosch",
    "Joan_Miro",
    "Kazimir_Malevich",
    "Leonardo_da_Vinci",
    "Marc_Chagall",
    "Mikhail_Vrubel",
    "Pablo_Picasso",
    "Paul_Gauguin",
    "Paul_Klee",
    "Peter_Paul_Rubens",
    "Pierre_Auguste_Renoir",
    "Pieter_Bruegel",
    "Raphael",
    "Rembrandt",
    "Rene_Magritte",
    "Salvador_Dali",
    "Sandro_Botticelli",
    "Titian",
    "Vincent_van_Gogh"
]

artists_dict = {name:index  for index, name in enumerate(artist_names)}

file_names = sorted([f for f in listdir("imgs") if f.endswith(".jpg")])

print(len(file_names))

random.shuffle(file_names)

train_files = file_names[:2424]
test_files = file_names[2424:]
check_files = file_names[151:200]

print(len(train_files))
print(len(test_files))

image_dir = "imgs"
new_size = (224,224)

train_list = [] 
train_label = [] # get name from file - artists_dict map the name to the number - e.g: index Durer - returns 0 


test_list = []
test_label = []

check_list = []
check_label = []

count = 0; 

for fname in test_files:
    a_name = "_".join(fname.split("_")[:-1])
    label = artists_dict.get(a_name, -1)

    if label != -1:
        img_space = path.join(image_dir, fname)
        new_img = open_image(img_space).resize(new_size)
        img_t = Tensor(new_img.pixels).reshape(224,224,-1).permute(2,0,1)
        if img_t.shape[0] == 1:
            img_t = img_t.repeat(3,1,1)
        


    test_list.append(img_t)
    test_label.append(label)



3030
2424
606


: 